In [1]:
from __future__ import unicode_literals,print_function

import json
import plac
import random
import spacy
import warnings
nlp_de1 = spacy.load("de_core_news_sm")
nlp_de2 = spacy.load("de_core_news_md")
SM_plus_Train_DATA_Model = spacy.load("ModelsRetrained/Novo_Models/UpdateModel1/RetrainModels/")
MD_plus_Train_Data_Model = spacy.load("/Users/zijian/ZijianStageEntiteNommee/Detect-Named-Entity-/spacyNER/ModelsRetrained/Novo_Models/UpdateModel2")

### Open data

In [3]:
with open("novo_train_de.json",'r',encoding="utf-8")as f:
    train_data = json.load(f)
with open("novo_dev_de.json",'r',encoding="utf-8")as f:
    dev_data = json.load(f)
print(train_data[0])
print(dev_data[0])

['Frankreich.', ['Frankreich', 0, 10, 'loc']]
['Rußland.', ['Rußland', 0, 7, 'loc']]


### Clean Data

In [5]:
def traiter_texte(text):
    text = text.replace("\n"," ").replace("\t"," ")
    text = text.replace("Institutin Zürich","Institut in Zürich")
    text = text.replace("desneuen"," des neuen")
    text = text.replace("delas","de las")
    text = text.replace("A. Z.)Belgien","A. Z.) Belgien")
    text = text.replace("derBompartschen","der Bompartschen")
    text = text.replace("aufZürich","auf Zürich")
    text = text.replace("derSociété","der Société")
    text = text.replace("denInsignien","den Insignien")
    return text

def label(label):
    label_map = {
        'pers':'PER',
        'org':'ORG',
        'loc':'LOC'
    }
    if label in label_map:
        return label_map[label]
    else:
        return "MISC"

def context(text,i,j):
    left,right = i,j
    count = 0
    if left > 0:
        while count < 5 and left > 0:
            left-=1
            if text[left]==' ':
                count+=1
    count = 0
    while count < 5 and right+1 < len(text):
        right += 1
        if text[right]==' ':
            count += 1
    return text[left:right]
    
for i in range(len(train_data)):
    train_data[i][0]=traiter_texte(train_data[i][0])
    for j in range(1,len(train_data[i])):
        train_data[i][j][3]=label(train_data[i][j][3])
        train_data[i][j].append(context(train_data[i][0],train_data[i][j][1],train_data[i][j][2]))
        
for i in range(len(dev_data)):
    dev_data[i][0]=traiter_texte(dev_data[i][0])
    for j in range(1,len(dev_data[i])):
        dev_data[i][j][3]=label(dev_data[i][j][3])
        dev_data[i][j].append(context(dev_data[i][0],dev_data[i][j][1],dev_data[i][j][2]))
print(train_data[0],len(train_data))
print(dev_data[0],len(dev_data))

['Frankreich.', ['Frankreich', 0, 10, 'MISC', 'Frankreich']] 104
['Rußland.', ['Rußland', 0, 7, 'LOC', 'Rußland']] 40


### find the NER

In [6]:
def context(text,i,j):
    left,right = i,j
    count = 0
    if left > 0:
        while count < 5 and left > 0:
            left-=1
            if text[left]==' ':
                count+=1
    count = 0
    while count < 5 and right+1 < len(text):
        right += 1
        if text[right]==' ':
            count += 1
    return text[left:right]

def consulter(model,dev):
    results = []
    for i in range(len(dev)):
        doc = model(dev[i][0])
        tmp = [dev[i][0]]
        for ent in doc.ents:
            tmp.append([ent.text,ent.start_char,ent.end_char,ent.label_,context(dev[i][0],ent.start_char,ent.end_char)])
        results.append(tmp) 
    return results

In [7]:
sm = consulter(nlp_de1,dev_data)
md = consulter(nlp_de2,dev_data)
sm_train = consulter(SM_plus_Train_DATA_Model,dev_data)
md_train = consulter(MD_plus_Train_Data_Model,dev_data)

### find the True positif

In [8]:
print(dev_data[0])
print(sm[0])
print(md[0])
print(sm_train[0])
print(md_train[0])

['Rußland.', ['Rußland', 0, 7, 'LOC', 'Rußland']]
['Rußland.', ['Rußland', 0, 7, 'LOC', 'Rußland']]
['Rußland.', ['Rußland', 0, 7, 'LOC', 'Rußland']]
['Rußland.', ['Rußland', 0, 7, 'LOC', 'Rußland']]
['Rußland.', ['Rußland', 0, 7, 'LOC', 'Rußland']]


In [58]:
def vp(res,correction):        
    VP = [] 
    FP,VN = [],[]
    for x in range(len(res)):
        a = res[x][1:]
        b = correction[x][1:]
        i,j = 0,0
        while i<len(a) and j<len(b):
            l1,r1 = a[i][1],a[i][2]
            l2,r2 = b[j][1],b[j][2]
            #print("modèle detection : ",a[i])
            #print("références : ",b[j])
            #print()
            if l1>r2:
                VN.append(b[j])
                j+=1
            elif r1 < l2:
                FP.append(a[i])
                i+=1
            elif l1==l2 and r1==r2:
                if a[i][3]!=b[j][3]:
                    a[i][3] += " but correct : ({})".format(b[j][3])
                    FP.append(a[i])
                else:
                    VP.append(a[i])
                i+=1
                j+=1
            else:
                if a[i][0] in b[j][4]:
                    if a[i][3]!=b[j][3]:
                        a[i][3] += " but correct : ({})".format(b[j][3])
                        FP.append(a[i])
                    else:
                        VP.append(a[i])
                else:
                    FP.append(a[i])
                i+=1
                j+=1  
            #print("Vrai positif : ",len(VP)," Faux positif ： ",len(FP)," Faux Négatif ： ",len(VN))
            #print("--------")
            #print()
        while i<len(a):
            FP.append(a[i])
            i+=1
        while j<len(b):
            VN.append(b[j])
            j+=1
    return [VP,FP,VN]
        
vp_sm,fp_sm,vn_sm = vp(sm,dev_data)[0]
vp_md = vp(md,dev_data)[0]
vp_sm_train = vp(sm_train,dev_data)[0]
vp_md_train = vp(md_train,dev_data)[0]
print(len(vp_sm)," ",len(vp_md)," ",len(vp_sm_train)," ",len(vp_md_train))

615   582   573   611


In [95]:
vp_sm[1]

['Petersburg',
 11,
 21,
 'LOC',
 '(Condeer.] Petersburg den 18. Dec. Se.russisch- kaiserl.']

In [97]:
c = 0
for x in vp_sm:
    if x[3]=='ORG':
        c+=1
print(c)

22


### Il y a trop de rappel pour les modèles existants en Spacy

In [86]:
vp1 = [tuple(x) for x in vp_sm]
vp2 = [tuple(x) for x in vp_md]
vp3 = [tuple(x) for x in vp_sm_train]
vp4 = [tuple(x) for x in vp_md_train]
print(len(vp1)," ",len(vp2)," ",len(vp3)," ",len(vp4))

615   582   573   611


In [105]:
intersection = list(set(vp3)&set(vp4))
print(len(intersection))

532


In [106]:
print(intersection[0])

('Marquis', 2205, 2212, 'PER', ' Darnleyund Craven dafür, der Marquis Lansdowne undder junge Lord Holland')


In [107]:
for x in intersection:
    print(x[4])
    print()

 Darnleyund Craven dafür, der Marquis Lansdowne undder junge Lord Holland

 Eidgenossen in der schönen Waadt! Mögeihrem Fest das schönste

 und drüngten zur Ernennung AhmedMahen Paschas als Nnchfolger, der als

 nichts einzuwenden, wenn der Bundesrath dem Polytechnikum, sowie denUniversitäten mit

 gans Deutschland und Oesterreich.Die Sowjetunion versucht, die Siege, die sieüber

 dem Weggange Bismarck's von Berlin vonder Szene verschwunden. Diese Taktik

 Hitze seines Charakters überlassen möchte.Hannover. Eine heftige Debatte in

 Löwen.Laut offiziellen Berichten des General Colaud, ebenfallsaus Brüssel vom 6.

 von Bagdad" Oper von Peter Cornelius; hierauf „Die Puppenfee", Ballett,

 Forsteru. Altorfer an der Theaterstraße, im Rütschihaus, findensich gegenwärtig

 hohen Preisen gewisserKonsumzentren, namentlich Zürichs, die Preisabreden schuld sind,

 Frankreichs, Hollands, Belgiens und Luxemburgs untereinander zu einer Politik gemeinsamer

 v. Wyß, Antistes Finsler, ForstmeisterMei

In [117]:
html_string1 = '''
<html>
  <meta charset="UTF-8"></meta>
  <head><title>Intersection</title></head>
  <style>
  <body>
    {table}
  </body>
</html>.
'''

import pandas as pd
#pd.set_option("display.max_colwidth",-1)
table = pd.DataFrame({"NER":[x[0] for x in intersection],
                      "Label":[x[3] for x in intersection],
                      "Context":[x[4] for x in intersection]
                     },index = [x for x in range(1,len(intersection)+1)])
table.head(5)



,NER,Label,Context
1,Marquis,PER,"Darnleyund Craven dafür, der Marquis Lansdowne undder junge Lord Holland"
2,Waadt,LOC,Eidgenossen in der schönen Waadt! Mögeihrem Fest das schönste
3,AhmedMahen,PER,"und drüngten zur Ernennung AhmedMahen Paschas als Nnchfolger, der als"
4,Bundesrath,ORG,"nichts einzuwenden, wenn der Bundesrath dem Polytechnikum, sowie denUniversitäten mit"
5,Sowjetunion,LOC,"gans Deutschland und Oesterreich.Die Sowjetunion versucht, die Siege, die sieüber"


In [116]:
with open("résultat/intersection2models.html","w",encoding='utf-8') as f:
    f.write(html_string1.format(table=table.to_html(border=1)))

In [121]:
table.to_html("1.html")